In [25]:
from IPython.core.display import display, HTML
display(HTML("<style>.container{max-width:80%!important;width:auto!important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import cv2

import numpy as np

import sys
sys.path.append('../')

from depth_Functions import (
    import_raw_colour_image, 
    show_img, 
    show_array_of_images
)
    
from Feature_Extraction import (
    process_patches, 
    create_local_feature_vector
)

# [Equation](https://www.cs.cornell.edu/~asaxena/learningdepth/ijcv_monocular3dreconstruction.pdf)

## Templates

Store variables for use in depicting equations

In [1]:
def symbol(symbol):
    return f'${symbol}$'

In [2]:
def equation(maths):
    return f'$${maths}$$'

In [3]:
V_feat = '\overline{f}_i'

In [4]:
V_theta = '\overline{\theta}_y'

In [5]:
V_rel = 'y_{ijs}'

In [6]:
V_v = '\overline{v}_y'

In [7]:
V_u = '\overline{u}_{ys}'

In [8]:
V_s1 = '\sigma_{1y}'

In [9]:
V_s2 = '\sigma_{2ys}'

In [10]:
S_row = "\sum_\limits{y=0}^Y"

In [11]:
S_col = "\sum_\limits{x=0}^X"

In [12]:
S_patch = f"{S_row}{S_col}"

In [13]:
S_adj = "\sum_\limits{j}^4" 

In [14]:
S_scale = "\sum_\limits{s=1}^3"

In [15]:
E_pred = f'{V_feat}{V_theta}'

In [16]:
E_local = f'\left( d_i - {E_pred} \right)^2'

In [17]:
E_global = '\left( d_i(s) - d_j(s) \right)^2'

In [18]:
E_s1 = f'{V_v}{V_feat}'

In [19]:
E_s2 = f'{V_u}|{V_rel}|'

In [20]:
E_e1 = f'\dfrac{{{E_local}}}{{2{V_s1}^2}}'

In [21]:
E_e2 = f'\dfrac{{{E_global}}}{{2{V_s2}^2}}'

In [22]:
E_gaus = f'\dfrac{{1}}{{Z}}e^\left(-{S_patch}{E_e1} - {S_scale}{S_patch}{S_adj}{E_e2} \right)'

In [23]:
E_log = f'-{E_e1} - {S_scale}{S_adj}{E_e2}'

In [24]:
E_squares = f'd_i = {E_pred} - 2{V_s1}^2{S_scale}{S_adj}{E_e2}'

## Lapasian

### Local accuracy

{{equation(S_patch+E_local)}}

Y = the number of vertical patches

X = the number of horizontal

$d_i$ = the depth at patch i

{{symbol(V_theta)}} = $\theta$ vector at row y

### Smoothing

To help smooth the model we take away the sum difference between the current patch and its neighbours at each scale

{{equation(S_scale+S_patch+S_adj+E_global)}}

$d_i(s)$ is the depth of the center patch at of the patch resized to scale s

$d_j(s)$ is the depth at the given adjacent patch at scale s

# Training

Having calculated the features for our training images the goal of training is to calculate $\theta$s and $\sigma$s

### Uncertanties

To combine the global and local features, we must encorporate uncertanties for both

#### Local uncertanty

{{equation(f'{V_s1}^2={E_s1}')}}

{{symbol(V_v)}} = vector of chosen parameters

#### Global uncertainty

{{equation(f'{V_s2}^2={E_s2}')}}

{{symbol(V_u)}} = vector of chosen parameters

{{symbol(V_rel)}} = relative depth histogram

## $\theta$

We calculate {{symbol(V_theta)}}s to be values that maximise $\log{P(d|X;\theta)}$ :

{{equation(f'{E_log} = 0')}}

We rearrange this to take the form y = mx + c for our least squares calculation:

{{equation(E_squares)}}